**Libs**:  
Scrapy | cartodb | numpy/pandas/matplotlib/scikit-learn | scikit-image | nltk | (plotly/graphene/etc)  

**Tools**:  
Python | firefox+firebug+firepath | Linux+CLI  

**Pages**:  
Idealista | Segundamano | ?  

**Time**:  
1h50m + 10m (Q/A)  

**Demos**:  
Scrapy | XPATH | panda | matplotlib | S.GIS (Filtering/etc) | CLI |  

# Siri, ¡Búscame un piso!
### Cómo encontrar el piso perfecto en <2h

Yuri Numerov (Achifaifa)  
[achi.se](http://achi.se)  -  [achi (at) hush (.) ai]()

Euskal Encounter 24, 2015-07-XX

### Sobre mí

¯\\\_(ツ)\_/¯

### Antes, un poco de spam...

*logo gamaker*

### Y otro poco más:

*pyladies* / *grupos regionales* / etc

### Analicemos el problema.

¿Cómo puede buscar alguien un piso?

#### A) Manera tradicional

* Abrir X pestañas en el navegador con diferentes páginas web

* Buscar por ciudad/CP

* Hacer scroll hasta que

* * Encontremos un piso

* * Nos aburramos 

* * Suframos de Síndrome de Túnel Carpiano

#### B) Manera **guay** 

* Escribimos un programa que lo haga por nosotros

* ???

* Profit

#### ¿A ó B?

matrix_pills.png

### ¿Qué herramientas vamos a usar?

#### Herramientas generales:

* [Python](http://python.org)

* Linux (CLI)

* iceweasel (Firefox) + firebug + firepath

* [CartoDB](https://cartodb.com): Servicio de visualización de datos cartográficos

#### Librerías de python:

* [Scrapy](http://scrapy.org): Librería para web scraping

* [Matplotlib](http://matplotlib.org): Librería para generación de gráficos

* [Numpy](http://www.numpy.org): Librería con *muchas* funciones matemáticas extra

* [Pandas](http://pandas.pydata.org): Librería para análisis y manipulación de datos

scikit-learn | scikit-image | nltk 

### ¿Cuál es el plan?

* Localizar webs con suficientes datos disponibles

* Recoger todos los datos de esa web

* Completar los datos

* Filtrar los datos, ordenarlos y visualizarlos

01) Localizando páginas web

02) Scrapy y XPATH

03) Escribiendo una araña

04) Aclarar y repetir para el resto de webs

05) Introducción a datos en CSV

06) Introducción a GIS (Filtrando por CP)

07) Primera visualización (Introducción a cartodb)

08) Datos sucios (Completando GPS)

09) Datos conseguidos. ¿Qué buscamos? (Piscina, jardín, precio...)

10) Asignando pesos a variables

11) Pandas, numpy, matplotlib: Primera tentativa

12) Introducción a machine learning (Datos de tecreros)

13) Resultados, mejora, etc

Conclusiones

Consejos, fuentes, información extra, etc

Dudas, preguntas

Despedida